In [1]:
import torch
from torch import nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In this notebook I will use linear network and convolutional network to recognise real-world data from FashionMNIST dataset.

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Now I'll pass Dateset as an argument to DataLoader. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading.

Let's define a batch size of 64.

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape, y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: (torch.Size([64]), torch.int64)


Creating Models. To define a neural network in PyTorch, we create a class that inherits from nn.Module. In the forward function we specify how data will pass through the network.

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
for param in model.parameters():
    print(param.numel())

401408
512
262144
512
5120
10


To train a model we need a loss function and an optimizer.

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [8]:
len(train_dataloader.dataset)

60000

In [12]:
def train(dataloader, model, criterion, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        
        # compute prediction error
        pred = model(X)
        loss = criterion(pred, y)
        
        # backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch*len(X)
            print(f"Loss: {loss:>7f} [{current:>5d}/{size:>5d}]")
            

In [10]:
def test(dataloader, model, criterion):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += criterion(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
import time

start_time = time.time()

epochs = 5

for t in range(epochs):
    print(f"Epoch {t+1}\n------------------------")
    train(train_dataloader, model, criterion, optimizer)
    test(test_dataloader, model, criterion)
print(f"Training took: {(time.time() - start_time)/60}min")

Epoch 1
------------------------
Loss: 0.177260 [    0/60000]
Loss: 0.224940 [ 6400/60000]
Loss: 0.208950 [12800/60000]
Loss: 0.225784 [19200/60000]
Loss: 0.332405 [25600/60000]
Loss: 0.249275 [32000/60000]
Loss: 0.242515 [38400/60000]
Loss: 0.326356 [44800/60000]
Loss: 0.262572 [51200/60000]
Loss: 0.296212 [57600/60000]
Test Error: 
 Accuracy: 87.4%, Avg loss: 0.352767 

Epoch 2
------------------------
Loss: 0.177234 [    0/60000]
Loss: 0.183591 [ 6400/60000]
Loss: 0.181679 [12800/60000]
Loss: 0.224586 [19200/60000]
Loss: 0.268006 [25600/60000]
Loss: 0.245281 [32000/60000]
Loss: 0.239744 [38400/60000]
Loss: 0.318824 [44800/60000]


KeyboardInterrupt: 